In [1]:
import altair as alt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('Train.csv', index_col='ID')
df.head()

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
ID,,,,,,,,,,,
1,D,Flight,4,2,177,3,low,F,44,1233,1
2,F,Flight,4,5,216,2,low,M,59,3088,1
3,A,Flight,2,2,183,4,low,M,48,3374,1
4,B,Flight,3,3,176,4,medium,M,10,1177,1
5,C,Flight,2,2,184,3,medium,F,46,2484,1


Dataset Information:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10999 entries, 1 to 10999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Warehouse_block      10999 non-null  object
 1   Mode_of_Shipment     10999 non-null  object
 2   Customer_care_calls  10999 non-null  int64 
 3   Customer_rating      10999 non-null  int64 
 4   Cost_of_the_Product  10999 non-null  int64 
 5   Prior_purchases      10999 non-null  int64 
 6   Product_importance   10999 non-null  object
 7   Gender               10999 non-null  object
 8   Discount_offered     10999 non-null  int64 
 9   Weight_in_gms        10999 non-null  int64 
 10  Reached.on.Time_Y.N  10999 non-null  int64 
dtypes: int64(7), object(4)
memory usage: 1.0+ MB


Finding out the Null Values:

In [4]:
df.isna().sum()

Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
Reached.on.Time_Y.N    0
dtype: int64

Encoding the categorical variables

In [5]:
from sklearn.preprocessing import LabelEncoder

Label_enc = LabelEncoder()

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = Label_enc.fit_transform(df[col])

In [6]:
df = df.drop(['Gender'], axis=1 )
df.head()

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
ID,,,,,,,,,,
1,3,0,4,2,177,3,1,44,1233,1
2,4,0,4,5,216,2,1,59,3088,1
3,0,0,2,2,183,4,1,48,3374,1
4,1,0,3,3,176,4,2,10,1177,1
5,2,0,2,2,184,3,2,46,2484,1



Defining the X and Y attributes


In [7]:
True_val_Reached_on_T = df['Reached.on.Time_Y.N'].value_counts()
type(True_val_Reached_on_T)


pandas.core.series.Series

In [8]:
X_attribute = df.drop(['Reached.on.Time_Y.N'], axis=1)
y_attribute = df['Reached.on.Time_Y.N']

We use StandardScaler to scale the data to a uniform usabale form

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_attribute), columns=X_attribute.columns)
print(X_scaled)

       Warehouse_block  Mode_of_Shipment  Customer_care_calls  \
0             0.447189         -2.004158            -0.047711   
1             1.118034         -2.004158            -0.047711   
2            -1.565345         -2.004158            -1.799887   
3            -0.894500         -2.004158            -0.923799   
4            -0.223656         -2.004158            -1.799887   
...                ...               ...                  ...   
10994        -1.565345          0.638342            -0.047711   
10995        -0.894500          0.638342            -0.047711   
10996        -0.223656          0.638342             0.828377   
10997         1.118034          0.638342             0.828377   
10998         0.447189          0.638342            -1.799887   

       Customer_rating  Cost_of_the_Product  Prior_purchases  \
0            -0.700755            -0.690722        -0.372735   
1             1.421578             0.120746        -1.029424   
2            -0.700755     

In [10]:
X = X_scaled
X.head()

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Discount_offered,Weight_in_gms
0,0.447189,-2.004158,-0.047711,-0.700755,-0.690722,-0.372735,-0.548034,1.889983,-1.468240
1,1.118034,-2.004158,-0.047711,1.421578,0.120746,-1.029424,-0.548034,2.815636,-0.333893
2,-1.565345,-2.004158,-1.799887,-0.700755,-0.565881,0.283954,-0.548034,2.136824,-0.159002
3,-0.894500,-2.004158,-0.923799,0.006689,-0.711529,0.283954,1.035735,-0.208162,-1.502484
4,-0.223656,-2.004158,-1.799887,-0.700755,-0.545074,-0.372735,1.035735,2.013404,-0.703244


In [11]:

from sklearn.utils import class_weight
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(X_attribute, y_attribute, test_size=0.2, random_state=42)

weights_for_class = class_weight.compute_class_weight('balanced', 
                                                  classes=np.unique(y_train), 
                                                  y=y_train)

dict_for_class_weight = dict(enumerate(weights_for_class))

In [12]:
print(dict_for_class_weight)

{0: 1.2424456368257555, 1: 0.8367249904906808}


Using GradientBosstingClassifier model from sklearn for the prediction purpose.

* we alsoCalculate the accuracy of the model on the training and testing sets and display them


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight


X_train, X_test, y_train, y_test = train_test_split(X_attribute, y_attribute, test_size=0.4, random_state=42)


weights_for_class = class_weight.compute_sample_weight('balanced', y_train)

gradient_boost_model = GradientBoostingClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features=None,
    subsample=1.0,
    random_state=42
)


gradient_boost_model.fit(X_train, y_train, sample_weight= weights_for_class)


y_train_data_prediction_gbc = gradient_boost_model.predict(X_train)
y_test_data_prediction_gbc = gradient_boost_model.predict(X_test)

train_data_acc = accuracy_score(y_train, y_train_data_prediction_gbc)
test_data_acc = accuracy_score(y_test, y_test_data_prediction_gbc)
print('The Train data accuracy by our model is : ', train_data_acc)
print('Test Test data accuracy by our model is : ', test_data_acc)

The Train data accuracy by our model is :  0.7279890892559479
Test Test data accuracy by our model is :  0.6915909090909091


We generate the classification report for the Gradient boost model

In [37]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_test_pred_gbc))

              precision    recall  f1-score   support

           0       0.57      0.93      0.71      1738
           1       0.93      0.53      0.68      2662

    accuracy                           0.69      4400
   macro avg       0.75      0.73      0.69      4400
weighted avg       0.78      0.69      0.69      4400

